In [1]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

import clingo
from IPython.display import clear_output

In [2]:
# Some machinery to (dynamically) print things in the notebook
log_str = ""
def reset_log():
    global log_str
    log_str = ""
def log(msg):
    global log_str
    print(msg, flush=True)
    log_str += f"{msg}\n"
def print_log():
    global log_str
    print(log_str, end='', flush=True)

def display_solution(model):
    print(model)


def find_and_output_solution(
    asp_program,
    timeout=10,
    num_threads=1,
):
    """
    Calls the ASP solver to find answer sets,
    and translates these to solutions,
    which are printed in human-readable format.
    """

    reset_log()
    #log("..Grounding..")
    
    control = clingo.Control([f"--parallel-mode={num_threads}"])
    control.add("base", [], asp_program)
    control.ground([("base", [])])

    control.configuration.solve.models = 0
    control.configuration.solve.opt_mode = "optN"
    
    models = []

    def interpret_model(model):
        #clear_output(wait=True)
        #print_log()
        #reset_log()
        #print(display_solution(model))
        save_mod = str(model).replace(' ', ' ')
        models.append(save_mod)


    if timeout > 0:
        #log(f"..Solving with timeout {timeout}s..")
        with control.solve(
            async_=True,
            on_model=lambda model: interpret_model(model),
        ) as handle:
            finished = handle.wait(timeout)
            if not finished:
                log("..Stopped after timeout..")
            else:
                #log("..Stopped..")
                time = control.statistics['summary']['times']['total']
                #log(f"..Total solving time: {time:.2f} seconds..")
    else:
        #log("..Solving..")
        control.solve(
            on_model=lambda model: interpret_model(model),
        )
        #log("..Stopped..")
        time = control.statistics['summary']['times']['total']
        #log(f"..Total solving time: {time:.2f} seconds..")

    return models

In [31]:
with open('get_graphs.lp', 'r') as file:
    get_graphs = file.read()

with open('enum_graphs.lp', 'r') as file:
    enum_graphs = file.read()

with open('find_rule.lp', 'r') as file:
    find_rule = file.read()
#print(get_graphs)

In [25]:
n_nodes=3
consts = f"#const num_nodes={n_nodes}.\n"
prog = consts+get_graphs
#print(prog)
edge_out = find_and_output_solution(prog, timeout=60, num_threads=4)
print(edge_out)
print("total graphs:", len(edge_out))

['edge(1,1) edge(2,1) edge(2,2) edge(3,1) edge(3,2) edge(3,3)', 'edge(1,1) edge(2,1) edge(2,2) edge(2,3) edge(3,1) edge(3,3)', 'edge(1,1) edge(2,1) edge(2,2) edge(2,3) edge(3,1) edge(3,2) edge(3,3)', 'edge(1,1) edge(1,3) edge(2,1) edge(2,2) edge(2,3) edge(3,3)', 'edge(1,1) edge(1,3) edge(2,1) edge(2,2) edge(2,3) edge(3,1) edge(3,3)', 'edge(1,1) edge(1,2) edge(2,2) edge(3,1) edge(3,2) edge(3,3)', 'edge(1,1) edge(1,2) edge(2,1) edge(2,2) edge(3,1) edge(3,2) edge(3,3)', 'edge(1,1) edge(1,2) edge(1,3) edge(2,2) edge(2,3) edge(3,3)', 'edge(1,1) edge(1,2) edge(1,3) edge(2,1) edge(2,2) edge(2,3) edge(3,3)', 'edge(1,1) edge(1,2) edge(1,3) edge(2,2) edge(3,2) edge(3,3)', 'edge(1,1) edge(1,2) edge(1,3) edge(2,2) edge(3,1) edge(3,2) edge(3,3)', 'edge(1,1) edge(1,2) edge(1,3) edge(2,1) edge(2,2) edge(2,3) edge(3,1) edge(3,2) edge(3,3)', 'edge(1,1) edge(1,2) edge(1,3) edge(2,2) edge(2,3) edge(3,2) edge(3,3)']
total graphs: 13


In [6]:
def out_to_asp(out):
    return out.replace(' ', '. \n') + '.\n'

graph_out = []
for i, out in enumerate(edge_out):
    prog = f"""#const graph_num={i+1}.\n"""
    edges = out_to_asp(out)
    prog += edges
    prog += enum_graphs
    #print(prog)
    num_out = find_and_output_solution(prog)[0]
    reset_log()
    graph_out.append(num_out)

graph_asp = ""
for out in graph_out:
    graph_asp += out_to_asp(out)

print(graph_asp)

graph(1,edge(1,1)). 
graph(1,edge(2,1)). 
graph(1,edge(2,2)). 
graph(1,edge(3,1)). 
graph(1,edge(3,2)). 
graph(1,edge(3,3)).
graph(2,edge(1,1)). 
graph(2,edge(2,1)). 
graph(2,edge(2,2)). 
graph(2,edge(2,3)). 
graph(2,edge(3,1)). 
graph(2,edge(3,3)).
graph(3,edge(1,1)). 
graph(3,edge(2,1)). 
graph(3,edge(2,2)). 
graph(3,edge(2,3)). 
graph(3,edge(3,1)). 
graph(3,edge(3,2)). 
graph(3,edge(3,3)).
graph(4,edge(1,1)). 
graph(4,edge(1,3)). 
graph(4,edge(2,1)). 
graph(4,edge(2,2)). 
graph(4,edge(2,3)). 
graph(4,edge(3,3)).
graph(5,edge(1,1)). 
graph(5,edge(1,3)). 
graph(5,edge(2,1)). 
graph(5,edge(2,2)). 
graph(5,edge(2,3)). 
graph(5,edge(3,1)). 
graph(5,edge(3,3)).
graph(6,edge(1,1)). 
graph(6,edge(1,2)). 
graph(6,edge(1,3)). 
graph(6,edge(2,2)). 
graph(6,edge(2,3)). 
graph(6,edge(3,3)).
graph(7,edge(1,1)). 
graph(7,edge(1,2)). 
graph(7,edge(1,3)). 
graph(7,edge(2,1)). 
graph(7,edge(2,2)). 
graph(7,edge(2,3)). 
graph(7,edge(3,3)).
graph(8,edge(1,1)). 
graph(8,edge(1,2)). 
graph(8,edge(1,3)). 

In [11]:
from itertools import permutations
def generate_profiles(n_agents=3,n_graphs=13):
    #n_graphs=13
    
    perms = list(permutations(range(1,n_graphs+1),n_agents))
    for i in range(1,n_graphs+1):
        perms += list([i] * n_agents)
    
    return perms

all_perms = generate_profiles()
print(all_perms)

[(1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 2, 6), (1, 2, 7), (1, 2, 8), (1, 2, 9), (1, 2, 10), (1, 2, 11), (1, 2, 12), (1, 2, 13), (1, 3, 2), (1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 3, 7), (1, 3, 8), (1, 3, 9), (1, 3, 10), (1, 3, 11), (1, 3, 12), (1, 3, 13), (1, 4, 2), (1, 4, 3), (1, 4, 5), (1, 4, 6), (1, 4, 7), (1, 4, 8), (1, 4, 9), (1, 4, 10), (1, 4, 11), (1, 4, 12), (1, 4, 13), (1, 5, 2), (1, 5, 3), (1, 5, 4), (1, 5, 6), (1, 5, 7), (1, 5, 8), (1, 5, 9), (1, 5, 10), (1, 5, 11), (1, 5, 12), (1, 5, 13), (1, 6, 2), (1, 6, 3), (1, 6, 4), (1, 6, 5), (1, 6, 7), (1, 6, 8), (1, 6, 9), (1, 6, 10), (1, 6, 11), (1, 6, 12), (1, 6, 13), (1, 7, 2), (1, 7, 3), (1, 7, 4), (1, 7, 5), (1, 7, 6), (1, 7, 8), (1, 7, 9), (1, 7, 10), (1, 7, 11), (1, 7, 12), (1, 7, 13), (1, 8, 2), (1, 8, 3), (1, 8, 4), (1, 8, 5), (1, 8, 6), (1, 8, 7), (1, 8, 9), (1, 8, 10), (1, 8, 11), (1, 8, 12), (1, 8, 13), (1, 9, 2), (1, 9, 3), (1, 9, 4), (1, 9, 5), (1, 9, 6), (1, 9, 7), (1, 9, 8), (1, 9, 10), (1, 9, 11), (1, 9, 12), (1, 9, 13),

In [44]:

import itertools
def generate_profiles(n_agents=3, n_graphs=13):
    g_range = list((range(1,n_graphs+1)))
    perms = [p for p in itertools.product(g_range, repeat=3)]
    #print(g_range)
    #print(perms)
    
    prog = f"""#const num_agents={n_agents}. #const num_graphs={n_graphs}. #const num_profiles={len(perms)}.\n""" + consts
    for i, p in enumerate(perms):
        if len(p) != n_agents:
            continue

        profile = i + 1
        for j, g in enumerate(p):
            agent = j + 1
            clause = f"profile({profile}, choose({agent},{g})).\n"

            prog += clause

    return prog

prof_asp = generate_profiles()

print(prof_asp)

#const num_agents=3. #const num_graphs=13. #const num_profiles=2197.
#const num_nodes=3.
profile(1, choose(1,1)).
profile(1, choose(2,1)).
profile(1, choose(3,1)).
profile(2, choose(1,1)).
profile(2, choose(2,1)).
profile(2, choose(3,2)).
profile(3, choose(1,1)).
profile(3, choose(2,1)).
profile(3, choose(3,3)).
profile(4, choose(1,1)).
profile(4, choose(2,1)).
profile(4, choose(3,4)).
profile(5, choose(1,1)).
profile(5, choose(2,1)).
profile(5, choose(3,5)).
profile(6, choose(1,1)).
profile(6, choose(2,1)).
profile(6, choose(3,6)).
profile(7, choose(1,1)).
profile(7, choose(2,1)).
profile(7, choose(3,7)).
profile(8, choose(1,1)).
profile(8, choose(2,1)).
profile(8, choose(3,8)).
profile(9, choose(1,1)).
profile(9, choose(2,1)).
profile(9, choose(3,9)).
profile(10, choose(1,1)).
profile(10, choose(2,1)).
profile(10, choose(3,10)).
profile(11, choose(1,1)).
profile(11, choose(2,1)).
profile(11, choose(3,11)).
profile(12, choose(1,1)).
profile(12, choose(2,1)).
profile(12, choose(3,12)).

In [32]:
out = find_and_output_solution(graph_asp + prof_asp + find_rule, timeout=600)
print(out)

RuntimeError: parsing failed